In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
from functools import partial

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizer, AutoTokenizer
from transformer_lens import HookedTransformer
import torch.nn.functional as F

In [3]:
from eap_ig.graph import Graph
from eap_ig.evaluate import evaluate_graph, evaluate_baseline
from eap_ig.attribute import attribute 

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

def load_model(adapter_path, hf_model_name, translens_model_name, scratch_cache_dir = None):
    base_model = AutoModelForCausalLM.from_pretrained(hf_model_name, cache_dir=scratch_cache_dir)
    model_with_lora = PeftModel.from_pretrained(base_model, adapter_path)
    model_with_lora = model_with_lora.merge_and_unload()
    model = HookedTransformer.from_pretrained(model_name=translens_model_name, hf_model=model_with_lora, cache_dir=scratch_cache_dir)  

    model.cfg.use_split_qkv_input = True
    model.cfg.use_attn_result = True
    model.cfg.use_hook_mlp_in = True
    model.cfg.ungroup_grouped_query_attention = True
    return model

In [5]:
def collate_EAP(xs):
    clean, corrupted, labels = zip(*xs)
    clean = list(clean)
    corrupted = list(corrupted)
    return clean, corrupted, labels

class EAPDataset(Dataset):
    def __init__(self, filepath):
        self.df = pd.read_csv(filepath)

    def __len__(self):
        return len(self.df)
    
    def shuffle(self):
        self.df = self.df.sample(frac=1)

    def head(self, n: int):
        self.df = self.df.head(n)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        return row['clean'], row['corrupted'], row['label']
    
    def to_dataloader(self, batch_size: int):
        return DataLoader(self, batch_size=batch_size, collate_fn=collate_EAP)
    
def get_logit_positions(logits: torch.Tensor, input_length: torch.Tensor):
    batch_size = logits.size(0)
    idx = torch.arange(batch_size, device=logits.device)

    logits = logits[idx, input_length - 1]
    return logits

def kl_divergence(logits: torch.Tensor, clean_logits: torch.Tensor, input_length: torch.Tensor, labels: torch.Tensor, mean=True, loss=True):
    logits = get_logit_positions(logits, input_length)
    clean_logits = get_logit_positions(clean_logits, input_length)

    probs = torch.softmax(logits, dim=-1)
    clean_probs = torch.softmax(clean_logits, dim=-1)

    results = F.kl_div(probs.log(), clean_probs.log(), log_target=True, reduction='none').mean(-1)
    return results.mean() if mean else results

In [6]:
scratch_cache_dir = "/mnt/fast0/rje41/.cache/huggingface"    
hf_model_name = "EleutherAI/pythia-1.4B-deduped"
translens_model_name="pythia-1.4B-deduped"
adapter_path = "../fine-tuning/add_sub/lora_opt_results/r32a64/checkpoint-282"
model = load_model(
        adapter_path=adapter_path,
        hf_model_name=hf_model_name,
        translens_model_name=translens_model_name,
        scratch_cache_dir=scratch_cache_dir,
    )

Loaded pretrained model pythia-1.4B-deduped into HookedTransformer


In [7]:
ds = EAPDataset('../datasets/Add_Sub_100_circuit.csv')
dataloader = ds.to_dataloader(6)

In [8]:
g = Graph.from_model(model)
attribute(model, g, dataloader, partial(kl_divergence, loss=True, mean=True), method='EAP-IG', ig_steps=5)

100%|██████████| 231493/231493 [00:00<00:00, 640825.59it/s]


In [9]:
total_edges = len(g.edges)
five_percent_edges = int(total_edges * 0.05)
g.apply_topn(five_percent_edges , absolute=True)
g.prune_dead_nodes()
g.to_json('graph.json')

In [10]:
import torch
from functools import partial

def calculate_faithfulness(model, g, dataloader, metric_fn):
    baseline_performance = evaluate_baseline(model, dataloader, metric_fn).mean().item()
    circuit_performance = evaluate_graph(model, g, dataloader, metric_fn).mean().item()
    faithfulness = abs(baseline_performance - circuit_performance)
    percentage_performance = (1 - faithfulness / baseline_performance) * 100

    print(f"Baseline performance: {baseline_performance}")
    print(f"Circuit performance: {circuit_performance}")
    print(f"Faithfulness: {faithfulness}")
    print(f"Percentage of model performance achieved by the circuit: {percentage_performance:.2f}%")

    return faithfulness, percentage_performance

metric_fn = partial(kl_divergence, loss=False, mean=False)
faithfulness, percentage_performance = calculate_faithfulness(model, g, dataloader, metric_fn)

100%|██████████| 84/84 [00:40<00:00,  2.07it/s]

Baseline performance: 0.0003752674092538655
Circuit performance: 0.000333296979079023
Faithfulness: 4.197043017484248e-05
Percentage of model performance achieved by the circuit: 88.82%


In [11]:
def exact_match_accuracy(model, logits, corrupted_logits, input_lengths, labels):

    batch_size = logits.size(0)
    device = logits.device

    positions = input_lengths - 1 

    # Gather the logits at these positions
    last_logits = logits[torch.arange(batch_size), positions, :]  # [batch_size, vocab_size]

    # Get the predicted tokens
    predicted_tokens = last_logits.argmax(dim=-1)  # [batch_size]

    # Convert predicted tokens to strings
    predicted_strings = [model.to_string(token.item()).strip() for token in predicted_tokens]

    # Convert labels to strings
    labels_strings = []
    for i in range(batch_size):
        lab = labels[i]
        if isinstance(lab, torch.Tensor):
            lab = lab.item()
        labels_strings.append(str(lab).strip())

    # Compute correctness
    correct = []
    for pred_str, label_str in zip(predicted_strings, labels_strings):
        print(f'pred_str:{pred_str}, labeel_str:{label_str}')
        if pred_str == label_str:
            correct.append(1.0)
        else:
            correct.append(0.0)

    return torch.tensor(correct, device=device)

# Evaluate the baseline model
baseline_accuracy = evaluate_baseline(model, dataloader, partial(exact_match_accuracy, model)).mean().item()

# Evaluate the graph model
graph_accuracy = evaluate_graph(model, g, dataloader, partial(exact_match_accuracy, model)).mean().item()

print(f"Baseline model accuracy: {baseline_accuracy:.4f}; Graph model accuracy: {graph_accuracy:.4f}")

  1%|          | 1/84 [00:00<00:31,  2.61it/s]

pred_str:26, labeel_str:26
pred_str:8, labeel_str:8
pred_str:24, labeel_str:24
pred_str:165, labeel_str:165
pred_str:6, labeel_str:6
pred_str:131, labeel_str:131


  2%|▏         | 2/84 [00:00<00:30,  2.65it/s]

pred_str:131, labeel_str:131
pred_str:16, labeel_str:16
pred_str:20, labeel_str:20
pred_str:59, labeel_str:59
pred_str:63, labeel_str:63
pred_str:11, labeel_str:11


  4%|▎         | 3/84 [00:01<00:30,  2.63it/s]

...

pred_str:138, labeel_str:138
pred_str:112, labeel_str:112
pred_str:82, labeel_str:82
pred_str:60, labeel_str:60
pred_str:14, labeel_str:14
pred_str:139, labeel_str:139


 99%|█████████▉| 83/84 [00:41<00:00,  2.04it/s]

pred_str:122, labeel_str:122
pred_str:65, labeel_str:65
pred_str:51, labeel_str:51
pred_str:16, labeel_str:16
pred_str:38, labeel_str:38
pred_str:26, labeel_str:24


100%|██████████| 84/84 [00:41<00:00,  2.03it/s]

pred_str:106, labeel_str:106
pred_str:70, labeel_str:70
Baseline model accuracy: 0.9600; Graph model accuracy: 0.7100
